See the README.md for project description

From the 5-jupyter things article

First maintain your imports in one cell so you can quickly see dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Set up inline graphics

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

The data import is easy with pandas cause this is how it reads a csv.

We can cut the header of our cnee file with a simple `grep Cypress goodmouse-2017-10-28-1.dat > cnee.csv` then read that file

In [ ]:
trackpad = pd.read_csv("cnee.csv", header=None)

You can always bring up help for a methon with the `help()` function, like `help(pd.read_csv)`

Take a look at the data just to make sure its what we think it is

In [ ]:
trackpad

A the plot method is part of all data object types and is a quick way to look at the data.  A simple data point plot can be drawn with the 'o' style.

In [ ]:
trackpad[2].plot(style='o')

What we really need is to see this data in 2D since we are mapping a 2D surface.  This means we want the X values on the x-axis and Y as the y-axis.  Just looking at the raw data let's us see where all the trackpad was touched.

It doesnt' give us any insight into when or what kind of trajectories we developed here but this is where we can start to use our clustering algos to see how closely together the interaction occurred

In [ ]:
plt.scatter(trackpad[2],trackpad[3])

Let's look at some longer running input and see if more of the traces can be seen.

In [ ]:
tp2=pd.read_csv("cnee2.csv", header=None)

plt.scatter(tp2[2],tp2[3])